<a href="https://colab.research.google.com/github/GitMarco27/TMML/blob/main/Notebooks/012_pythonflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Minutes Machine Learning
## Episode 12: Pythonflow: from eager to graph python programming

#### Marco Sanguineti, 2021
---
Welcome to 3 minutes Machine Learning!

In [32]:
! pip install pythonflow -U -q

In [33]:
import tensorflow as tf
import pythonflow as pf
import numpy as np

# TensorflowGraph

In [34]:
class AddLayerOperation(pf.Operation):
    def __init__(self, model,  type_layers, num_nodes, activation):
        super(AddLayerOperation, self).__init__(model, type_layers, num_nodes, activation)


    def _evaluate(self, model, type_layers, num_nodes, activation):
        if type_layers=='Dense':
          new_layer = tf.keras.layers.Dense(num_nodes, activation=activation)

        model.add(new_layer)
        return model

class CompileOperation(pf.Operation):
    def __init__(self, model, number_name):
        super(CompileOperation, self).__init__(model, name='CompileOperation'+number_name)

    def _evaluate(self, model):
        model.compile(optimizer='adam', loss='mse')
        return model

class TrainOperation(pf.Operation):
    def __init__(self, model, epochs, batch_size, X, y):
        super(TrainOperation, self).__init__(model, epochs, batch_size,  X, y, name='TrainModel')

    def _evaluate(self, model,X, y,  epochs,batch_size):
        history = model.fit(X, y, epochs=epochs, batch_size=batch_size)
        return model, history

with pf.Graph() as graphadd:
    a = pf.constant(4)
    b = pf.constant(38)
    x = (a + b).set_name('x')

with pf.Graph() as graph:
    num_nodes = pf.placeholder(name="num_nodes")
    activation = pf.placeholder(name="activation")
    type_layers = pf.placeholder(name="type_layers")
    epochs = pf.placeholder(name="epochs")
    batch_size = pf.placeholder(name="batch_size")

    input_layer = pf.func_op(tf.keras.Input, [graphadd('x')])
    model = pf.func_op(tf.keras.models.Sequential, [input_layer], name='model')

    model_graph = AddLayerOperation(model, type_layers, num_nodes, activation)
    model_graph1 = AddLayerOperation(model_graph, type_layers, num_nodes, activation)
    model_graph2 = AddLayerOperation(model_graph1, type_layers, num_nodes, activation)
    model_graph_compile1 = CompileOperation(model_graph2, number_name='1')

    X = pf.func_op(np.random.rand, *[100, graphadd('x')])
    y = pf.func_op(np.random.rand, *(100, 10))

    model_trained = TrainOperation(model_graph_compile1, X, y, epochs, batch_size)


model = graph(model_graph_compile1 , num_nodes=10, activation='relu', type_layers='Dense', 
               batch_size= 16, epochs=5)

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 10)                430       
                                                                 
 dense_28 (Dense)            (None, 10)                110       
                                                                 
 dense_29 (Dense)            (None, 10)                110       
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [35]:
model, history = graph(model_trained , num_nodes=10, activation='relu', type_layers='Dense',
                       batch_size= 16, epochs=5)

Epoch 1/5
7/7 [==============================] - 0s 5ms/step - loss: 0.2173
Epoch 2/5
7/7 [==============================] - 0s 3ms/step - loss: 0.2011
Epoch 3/5
7/7 [==============================] - 0s 3ms/step - loss: 0.1940
Epoch 4/5
7/7 [==============================] - 0s 3ms/step - loss: 0.1886
Epoch 5/5
7/7 [==============================] - 0s 2ms/step - loss: 0.1836


In [36]:
import time
def create_graph(num_nodes=10, activation='relu', type_layers='Dense',batch_size= 16, epochs=5 ): 
  models = graph(model_trained, num_nodes=num_nodes, activation=activation, type_layers=type_layers, 
                 batch_size= batch_size, epochs=epochs)
  return models

In [37]:
import pickle 

with open("custom_graph", 'wb') as f:
  pickle.dump(graph, f)

In [38]:
with open("custom_graph", 'rb') as f:
  new_graph = pickle.load(f)

del model
model, history = new_graph('TrainModel', num_nodes=10, activation='selu', type_layers='Dense',
                           batch_size= 16, epochs=5)
model.summary()


Epoch 1/5
7/7 [==============================] - 0s 2ms/step - loss: 0.8081
Epoch 2/5
7/7 [==============================] - 0s 2ms/step - loss: 0.5746
Epoch 3/5
7/7 [==============================] - 0s 2ms/step - loss: 0.4853
Epoch 4/5
7/7 [==============================] - 0s 5ms/step - loss: 0.4442
Epoch 5/5
7/7 [==============================] - 0s 3ms/step - loss: 0.4065
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 10)                430       
                                                                 
 dense_34 (Dense)            (None, 10)                110       
                                                                 
 dense_35 (Dense)            (None, 10)                110       
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_____

In [39]:
vars(new_graph).keys()

dict_keys(['operations', 'dependencies'])

In [40]:
print(new_graph.operations)
print(new_graph.dependencies)

{'num_nodes': <pf.placeholder 'num_nodes'>, 'activation': <pf.placeholder 'activation'>, 'type_layers': <pf.placeholder 'type_layers'>, 'epochs': <pf.placeholder 'epochs'>, 'batch_size': <pf.placeholder 'batch_size'>, '9b632894402c44d8a350599df56d457c': <pf.func_op '9b632894402c44d8a350599df56d457c' target=<function Input at 0x7f2ad5afc9e0> args=<1 items> kwargs=<0 items>>, 'model': <pf.func_op 'model' target=<class 'keras.engine.sequential.Sequential'> args=<1 items> kwargs=<0 items>>, '303900c3a5ee474d9bd5cfc26105225c': <__main__.AddLayerOperation object at 0x7f2b51db8c90>, 'e77a8e676e754bdab36b4325b92b7c2c': <__main__.AddLayerOperation object at 0x7f2b51db8d50>, '427382c0a3aa44268e35c6455e9803f9': <__main__.AddLayerOperation object at 0x7f2b51db8e10>, 'CompileOperation1': <__main__.CompileOperation object at 0x7f2b51db8ed0>, 'd0e3e4b1659d43c5950f0a80db57f846': <pf.func_op 'd0e3e4b1659d43c5950f0a80db57f846' target=<built-in method rand of numpy.random.mtrand.RandomState object at 0x7

In [43]:
import threading
import time
import sys
import trace
import threading
import time

class thread_with_trace(threading.Thread):
  def __init__(self, *args, **keywords):
    threading.Thread.__init__(self, *args, **keywords)
    self.killed = False
 
  def start(self):
    self.__run_backup = self.run
    self.run = self.__run     
    threading.Thread.start(self)
 
  def __run(self):
    sys.settrace(self.globaltrace)
    self.__run_backup()
    self.run = self.__run_backup
 
  def globaltrace(self, frame, event, arg):
    if event == 'call':
      return self.localtrace
    else:
      return None
 
  def localtrace(self, frame, event, arg):
    if self.killed:
      if event == 'line':
        raise SystemExit()
    return self.localtrace
 
  def kill(self):
    self.killed = True

print('Actual count: ', threading.active_count())
th = thread_with_trace(target=create_graph, args=[10,'relu', 'Dense', 16, 10])
th.start()
print('Actual count: ', threading.active_count(), '\n')
time.sleep(10)
th.kill()
th.join()
print('\n', 'Actual count: ', threading.active_count())


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "<ipython-input-43-fe0ce331204c>", line 18, in __run
    sys.settrace(self.globaltrace)



Actual count:  10
Actual count:  11 

Epoch 1/10
7/7 [==============================] - 1s 4ms/step - loss: 0.2853
Epoch 2/10
7/7 [==============================] - 0s 4ms/step - loss: 0.2683
Epoch 3/10
7/7 [==============================] - 0s 4ms/step - loss: 0.2440
Epoch 4/10
7/7 [==============================] - 0s 4ms/step - loss: 0.2191
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 0.2026
Epoch 6/10
7/7 [==============================] - 0s 4ms/step - loss: 0.1966
Epoch 7/10
7/7 [==============================] - 0s 4ms/step - loss: 0.1879
Epoch 8/10
7/7 [==============================] - 0s 4ms/step - loss: 0.1792
Epoch 9/10
7/7 [==============================] - 0s 4ms/step - loss: 0.1731
Epoch 10/10
7/7 [==============================] - 0s 4ms/step - loss: 0.1693

 Actual count:  10
